<a href="https://colab.research.google.com/github/ahmadluay9/ADK-Training/blob/main/05_mcp_toolbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/genai-toolbox/blob/main/docs/en/getting-started/colab_quickstart.ipynb)

# Getting Started With MCP Toolbox

This guide demonstrates how to quickly run
[Toolbox](https://github.com/googleapis/genai-toolbox) end-to-end in Google
Colab using Python, Snowflake, and [Google
GenAI](https://pypi.org/project/google-genai/), [ADK](https://google.github.io/adk-docs/)

Within this Colab environment, you'll
- Launch a Toolbox server.
- Connect to Toolbox and develop a sample `Demo` application.



## Step 1: Install and configure Toolbox

In this section, we will
1. Download the latest version of the toolbox binary.
2. Create a toolbox config file.
3. Start a toolbox server using the config file.



Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [ ]:
version = "0.27.0" # x-release-please-version
! curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

In [ ]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

> Note: To include a literal dollar sign (e.g., $1) as part of your SQL statement within the Python string for tools.yml, you must escape both the backslash and the dollar sign. Use \\\$1 in Python to output \$1 in the tools.yml file.

> Note: You can also set up Colab secrets to store any sensitive information like passwords. You can easily add secrets through the left panel:

<img src="https://services.google.com/fh/files/misc/colab_secret.png" alt="Colab Secrets" width="400"/>


Create a tools file with the following functions:

- `Database Connection (sources)`: `Includes details for connecting to our Snowflake database.`
- `Tool Definitions (tools)`: `Defines ten tools for database interaction:`
  - `execute-snowflake-query`
  - `list-tables`
  - `search-patients`
  - `get-patient-medical-history`
  - `get-patient-lab-results`
  - `get-patient-billing-status`
  - `find-doctors-by-specialty`
  - `check-doctor-appointments`
  - `check-hospital-bed-availability`
  - `check-medication-stock`

Our application will leverage these tools to interact with the database.

For detailed configuration options, please refer to the [Toolbox documentation](https://googleapis.github.io/genai-toolbox/getting-started/configure/).



In [ ]:
import os
from google.colab import userdata

def load_colab_secrets() -> dict:
    """
    Load required Snowflake credentials from Colab Secrets
    into environment variables and return them as a dictionary.
    """

    required_keys = [
        "SNOWFLAKE_ACCOUNT",
        "SNOWFLAKE_USERNAME",
        "SNOWFLAKE_PASSWORD",
        "SNOWFLAKE_DATABASE",
        "SNOWFLAKE_SCHEMA",
        "SNOWFLAKE_WAREHOUSE",
        "SNOWFLAKE_ROLE"
    ]

    loaded_secrets = {}
    missing_keys = []

    print("🔄 Loading credentials from Colab Secrets...\n")

    for key in required_keys:
        try:
            value = userdata.get(key)

            if value is None or value == "":
                missing_keys.append(key)
                continue

            os.environ[key] = value
            loaded_secrets[key] = value
            print(f"✅ {key} loaded")

        except Exception as e:
            missing_keys.append(key)
            print(f"⚠️ Error loading {key}: {e}")

    if missing_keys:
        print("\n❌ Missing secrets:")
        for key in missing_keys:
            print(f"   - {key}")
        print("\nTip: Ensure 'Notebook access' is enabled in the Secrets panel.")

    else:
        print("\n🎉 All secrets successfully loaded.")

    return loaded_secrets

In [ ]:
secrets = load_colab_secrets()

SNOWFLAKE_ACCOUNT = secrets.get("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USERNAME = secrets.get("SNOWFLAKE_USERNAME")
SNOWFLAKE_PASSWORD = secrets.get("SNOWFLAKE_PASSWORD")
SNOWFLAKE_DATABASE = secrets.get("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = secrets.get("SNOWFLAKE_SCHEMA")
SNOWFLAKE_WAREHOUSE = secrets.get("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_ROLE = secrets.get("SNOWFLAKE_ROLE")

In [ ]:
# Create a tools file at runtime.
# You can also upload a tools file and use that to run toolbox.
tools_file_name = "tools.yml"
file_content = f"""
sources:
  # Snowflake Source Configuration
  eikon-snowflake-source:
    kind: snowflake
    # The account identifier is the subdomain from your URL: wrtigml-fo42291
    account: {SNOWFLAKE_ACCOUNT}
    user: {SNOWFLAKE_USERNAME}
    password: {SNOWFLAKE_PASSWORD}
    database: {SNOWFLAKE_DATABASE}
    schema: {SNOWFLAKE_SCHEMA}
    warehouse: {SNOWFLAKE_WAREHOUSE}
    role: {SNOWFLAKE_ROLE}

tools:
  # --- ADMIN / GENERAL TOOLS ---
  execute-snowflake-query:
    kind: snowflake-execute-sql
    source: eikon-snowflake-source
    description: >
      Directly execute any SQL statement against the EIKON database.
      Useful for complex joins, DDL, or exploring data structure not covered by specific tools.
      Do not use for production automated agents without review.

  list-tables:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: List all available tables in the database schema.
    statement: |
      SELECT TABLE_NAME, ROW_COUNT
      FROM INFORMATION_SCHEMA.TABLES
      WHERE TABLE_SCHEMA = CURRENT_SCHEMA();

  search-patients:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      Search for a patient list to get details like ID, DOB, and Gender.
      Useful when you need to confirm which "Siti" or "Budi" you are looking for.
    parameters:
      - name: search_term
        type: string
        description: The name or ID of the patient.
    statement: |
      SELECT PATIENT_ID, NAME, GENDER, BIRTH_YEAR, BPJS_NUMBER
      FROM PATIENTS
      WHERE TO_VARCHAR(PATIENT_ID) = :1 OR NAME ILIKE '%' || :1 || '%'
      LIMIT 10;

  get-patient-medical-history:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      Retrieve medical history (diagnoses).
      Accepts either a Patient ID (e.g., 1001) or a Patient Name (e.g., "Siti").
    parameters:
      - name: patient_identifier
        type: string
        description: The Patient ID OR Patient Name.
    statement: |
      SELECT
        p.NAME AS PATIENT_NAME,
        p.PATIENT_ID,
        mr.RECORD_ID,
        mr.ICD10_CODE,
        icd.C2 AS DIAGNOSIS_DESCRIPTION
      FROM MEDICAL_RECORDS mr
      JOIN PATIENTS p ON mr.PATIENT_ID = p.PATIENT_ID
      LEFT JOIN ICD10_CODES icd ON mr.ICD10_CODE = icd.C1
      WHERE TO_VARCHAR(p.PATIENT_ID) = :1 OR p.NAME ILIKE '%' || :1 || '%';

  get-patient-lab-results:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      Get laboratory test results.
      Accepts either a Patient ID or a Patient Name.
    parameters:
      - name: patient_identifier
        type: string
        description: The Patient ID OR Patient Name.
    statement: |
      SELECT
        p.NAME AS PATIENT_NAME,
        l.LAB_ID,
        l.TEST,
        l.RESULT,
        h.NAME AS HOSPITAL_NAME
      FROM LAB_RESULTS l
      JOIN PATIENTS p ON l.PATIENT_ID = p.PATIENT_ID
      LEFT JOIN HOSPITALS h ON l.HOSPITAL_ID = h.HOSPITAL_ID
      WHERE TO_VARCHAR(p.PATIENT_ID) = :1 OR p.NAME ILIKE '%' || :1 || '%';

  get-patient-billing-status:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      Get billing details including total amount, paid amount, and insurance coverage.
      Accepts either a Patient ID or a Patient Name.
    parameters:
      - name: patient_identifier
        type: string
        description: The Patient ID OR Patient Name.
    statement: |
      SELECT
        p.NAME AS PATIENT_NAME,
        b.BILL_ID,
        h.NAME as HOSPITAL_NAME,
        b.AMOUNT as TOTAL_BILL,
        b.PAID_AMOUNT,
        b.INSURANCE_COVERED,
        (b.AMOUNT - b.PAID_AMOUNT - b.INSURANCE_COVERED) as REMAINING_BALANCE
      FROM BILLING b
      JOIN PATIENTS p ON b.PATIENT_ID = p.PATIENT_ID
      JOIN HOSPITALS h ON b.HOSPITAL_ID = h.HOSPITAL_ID
      WHERE TO_VARCHAR(p.PATIENT_ID) = :1 OR p.NAME ILIKE '%' || :1 || '%';

  find-doctors-by-specialty:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: Find doctors based on their medical specialty.
    parameters:
      - name: specialty
        type: string
        description: The specialty to search for (e.g., 'Neurology', 'Pediatrics').
    statement: |
      SELECT d.DOCTOR_ID, d.NAME, d.SPECIALTY, h.NAME as HOSPITAL_NAME
      FROM DOCTORS d
      JOIN HOSPITALS h ON d.HOSPITAL_ID = h.HOSPITAL_ID
      WHERE d.SPECIALTY ILIKE :1;

  check-doctor-appointments:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      List scheduled appointments.
      Accepts either a Doctor ID (e.g., 101) or a Doctor Name (e.g., "Budi").
      **IMPORTANT**: Do not include titles or honorifics like "Dr." or "Doctor" in the name parameter.
    parameters:
      - name: doctor_identifier
        type: string
        description: The Doctor ID OR Doctor Name.
    statement: |
      SELECT
        d.NAME as DOCTOR_NAME,
        a.APPOINTMENT_ID,
        a.DATE,
        p.NAME as PATIENT_NAME
      FROM APPOINTMENTS a
      JOIN PATIENTS p ON a.PATIENT_ID = p.PATIENT_ID
      JOIN DOCTORS d ON a.DOCTOR_ID = d.DOCTOR_ID
      WHERE TO_VARCHAR(d.DOCTOR_ID) = :1 OR d.NAME ILIKE '%' || :1 || '%'
      ORDER BY a.DATE DESC;

  check-hospital-bed-availability:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: >
      Check the status of beds.
      Accepts either a Hospital ID (e.g., 1) or a Hospital Name (e.g., "Jakarta").
    parameters:
      - name: hospital_identifier
        type: string
        description: The Hospital ID OR Hospital Name.
    statement: |
      SELECT
        h.NAME as HOSPITAL_NAME,
        b.BED_TYPE,
        b.STATUS,
        COUNT(*) as TOTAL
      FROM BEDS b
      JOIN HOSPITALS h ON b.HOSPITAL_ID = h.HOSPITAL_ID
      WHERE TO_VARCHAR(h.HOSPITAL_ID) = :1 OR h.NAME ILIKE '%' || :1 || '%'
      GROUP BY h.NAME, b.BED_TYPE, b.STATUS;

  check-medication-stock:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: Check inventory levels for a specific medication name.
    parameters:
      - name: medication_name
        type: string
        description: The name of the medication (e.g., 'Paracetamol').
    statement: |
      SELECT m.NAME, m.MEDICATION_ID, i.STOCK
      FROM MEDICATIONS m
      JOIN PHARMACY_INVENTORY i ON m.MEDICATION_ID = i.MEDICATION_ID
      WHERE m.NAME ILIKE :1;

toolsets:
  # General database exploration
  admin_toolset:
    - execute-snowflake-query
    - list-tables

  patient_services_toolset:
    - search-patients
    - get-patient-medical-history
    - get-patient-lab-results
    - get-patient-billing-status

  hospital_ops_toolset:
    - find-doctors-by-specialty
    - check-doctor-appointments
    - check-hospital-bed-availability
    - check-medication-stock
"""

In [ ]:
# 1. Write the file safely using Python (Avoids shell escaping issues)
with open(tools_file_name, "w") as f:
    f.write(file_content)

In [ ]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [ ]:
!pkill -f toolbox

In [ ]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --ui --tools-file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &
print(f"✅ Toolbox started on port {SERVER_PORT}")

In [ ]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

In [ ]:
# Print logs if it fails to ensure we see the error
! sleep 2 && cat toolbox.log

## Step 3: Connect your agent to Toolbox

In this section, you will
1. Establish a connection to the tools by creating a Toolbox client.
2. Build an agent that leverages the tools and an LLM.


> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [ ]:
# Run this and allow access through the pop-up
from google.colab import auth

auth.authenticate_user()

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "your-project-id"  # @param {type:"string"}
location = 'us-central1' # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

> - [Connect using ADK](#scrollTo=QqRlWqvYNKSo)


### Connect Using ADK

In [ ]:
! pip install google-adk[toolbox] toolbox-core --upgrade --quiet

In [ ]:
# environment configuration
import os

try:
    # The SDK uses this ID for usage tracking and billing
    os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

    # Defines the region where Vertex AI resources are hosted
    os.environ['GOOGLE_CLOUD_LOCATION'] = location

    # Directs the SDK to use Vertex AI infrastructure instead of the public Gemini API
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = "1"

    print(f"✅ Environment configured for project: {project_id} in {location}")

except Exception as e:
    print(f"❌ Configuration Error: {e}")

In [ ]:
# Import components
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.toolbox_toolset import ToolboxToolset
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.runners import InMemoryRunner
from google.genai import types

from toolbox_core import ToolboxSyncClient

In [ ]:
# Configure Retry Options
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)


In [ ]:
server_url="http://127.0.0.1:5000"

toolbox = ToolboxSyncClient(server_url)

admin_toolset = toolbox.load_toolset('admin_toolset')
patient_services_toolset = toolbox.load_toolset('patient_services_toolset')
hospital_ops_toolset = toolbox.load_toolset('hospital_ops_toolset')

print("✅ MCP Tool created")

In [ ]:
admin_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    name='admin_agent',
    description='Useful for database administration tasks, listing tables, and executing raw SQL queries.',
    instruction='You are a Database Administrator. Use the available tools to explore the database schema or execute complex SQL queries that other agents cannot handle.',
    tools=admin_toolset
)

admin_agent_tool = AgentTool(agent=admin_agent, skip_summarization=False)

In [ ]:
hospital_agent  = LlmAgent(
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    name='hospital_ops_agent',
    description='Useful for hospital operations, including doctor searches, bed availability, and pharmacy inventory.',
    instruction='You are a Hospital Operations Manager. Assist with finding doctors by specialty, checking bed capacity, and verifying medication stock levels.',
    tools=hospital_ops_toolset
)

hospital_agent_tool  = AgentTool(agent=hospital_agent, skip_summarization=False)

In [ ]:
patient_agent   = LlmAgent(
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    name='patient_services_agent',
    description='Useful for retrieving patient details, medical history, lab results, and billing information.',
    instruction='You are a Patient Services Representative. Help users find patient records, check medical histories, and review billing status using the provided tools. Always ask for the patient name if not provided.',
    tools=patient_services_toolset
)

patient_agent_tool = AgentTool(agent=patient_agent, skip_summarization=False)

In [ ]:
structured_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    name='structured_data_agent',
    description='A specialist agent that handles all structured database interactions regarding patients, hospital operations, and administration.',
    instruction='You are a coordinator for structured database queries. Delegate the user request to the most appropriate specialist (Admin, Patient Services, or Hospital Operations).',
    tools=[
        admin_agent_tool,
        hospital_agent_tool,
        patient_agent_tool
    ],
    generate_content_config= types.GenerateContentConfig(
        temperature=0.1
        ),
    output_key='structured_agent_response'
)

In [ ]:
root_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    name='root_agent',
    description='A helpful assistant for hospital staff.',
    instruction='You are the main interface for the Hospital System. Route user questions to the structured data agent for database queries.',
    sub_agents=[
        structured_agent
    ]
)

In [ ]:
queries = [
        "Pull up the medical history for Patient ID: 3",
        "Get the laboratory results for patient Siti Hidayat Patient ID: 1",
        # "Find me a list of doctors who specialize in Neurology.",
        # "List all upcoming appointments for Dr. fajar susanto.",
        "Check the availability of ICU beds at RS Jakarta Pusat."
]

In [ ]:
import asyncio

for i, queries in enumerate(queries, 1):
    print(f"\n{'='*60}")
    print(f"Query {i}: {queries}")
    print(f"{'='*60}\n")

    try:
        runner = InMemoryRunner(agent=root_agent)
        # 3. Execute
        await runner.run_debug(queries, verbose=True)

        await asyncio.sleep(4)

    except Exception as e:
        print(f"\n❌ Error executing Test Case {i}: {e}")